In [10]:
import fitz
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelForCausalLM, AutoTokenizer
import os

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)
    for page in pdf_document:
        text += page.get_text()
    return text

# Function to translate text using BERT
def translate_text_with_bert(text, target_language="fr"):
    model_name = "bert-base-multilingual-cased"
    model = BertForSequenceClassification.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)

    # Translate the input text to the target language
    input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=128, truncation=True, padding=True, return_tensors="pt")
    target_language_code = tokenizer.get_vocab().get(target_language, tokenizer.get_vocab()["[UNK]"])
    translation_ids = model.generate(
        input_ids,
        num_beams=4,
        max_length=128,
        early_stopping=True,
        num_return_sequences=1,
        forced_bos_token_id=target_language_code
    )

    # Decode the translated text
    translated_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    return translated_text

# Function to create a new PDF document with translated text
def create_translated_pdf(input_pdf_path, output_pdf_path, target_language="fr"):
    extracted_text = extract_text_from_pdf(input_pdf_path)
    translated_text = translate_text_with_bert(extracted_text, target_language)

    pdf_document = fitz.open()
    pdf_document.insert_page(0, width=600, height=800)
    page = pdf_document[0]
    page.insert_text(translated_text)

    pdf_document.save(output_pdf_path)
    pdf_document.close()

# # Paths for input and output PDF files
# input_pdf_path = "your_input.pdf"
# output_pdf_path = "translated_outputBERT.pdf"

# # Translate and create the new PDF
# create_translated_pdf(input_pdf_path, output_pdf_path)

# print(f"Translation complete. Translated PDF saved at: {output_pdf_path}")


In [11]:
# Input and output file paths
input_pdf_path = "uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"
output_pdf_path = "translated_outputBERT.pdf"

# Translate and create the new PDF
create_translated_pdf(input_pdf_path, output_pdf_path)

print(f"Translation complete. Translated PDF saved at: {output_pdf_path}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: The current model class (BertForSequenceClassification) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'BertLMHeadModel'}